<a href="https://colab.research.google.com/github/laurieye/sharing-github/blob/main/XGboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving cleaned_train 2.csv to cleaned_train 2.csv


In [ ]:
import pandas as pd
import io

model_data = pd.read_csv(io.BytesIO(uploaded['cleaned_train 2.csv']))

In [ ]:
#Drop 1/3 cd=0 and duplicates 1/3 cd=1
num_to_drop_cd0 = int(len(model_data[model_data['cd'] == 0]) / 3)
model_data = model_data.drop(model_data[model_data['cd'] == 0].sample(num_to_drop_cd0).index)
num_to_duplicate_cd1 = int(len(model_data[model_data['cd'] == 1]) / 3)
cd1_rows = model_data[model_data['cd'] == 1]
model_data = pd.concat([model_data, cd1_rows.sample(num_to_duplicate_cd1)], ignore_index=True)

In [ ]:
#import pandas as pd
#import numpy as np


# Identify rows where "cd" is equal to 0
#rows_to_remove = model_data [model_data ['cd'] == 0].index

# Calculate the number of rows to remove (half of the rows with cd == 0)
#num_rows_to_remove = len(rows_to_remove) // 2

# Randomly select half of the rows to remove
#np.random.seed(42)  # You can set any seed value for reproducibility
#rows_to_remove = np.random.choice(rows_to_remove, num_rows_to_remove, replace=False)

# Remove the selected rows from the dataset
#model_data = model_data.drop(rows_to_remove)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Assuming data is already loaded and model_data DataFrame exists
X = model_data.drop(columns=['cd'])
y = model_data['cd']

# Convert columns to appropriate types
label_encoders = {}
columns_to_encode = ['default', 'housing', 'loan']
for col in columns_to_encode:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Over-sample using SMOTE
#sm = SMOTE(random_state=42)
#X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Convert data to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)



In [ ]:
# Define the hyperparameter search space
space = {
    'nrounds': hp.quniform('nrounds', 50, 150, 1),
    'eta': hp.uniform('eta', 0.01, 0.3),
    'max_depth': hp.choice('max_depth', range(3, 11)),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
    #'alpha': hp.uniform('alpha', 0, 5)
}

# Compute scale_pos_weight for class imbalance
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)

def objective(params):
    num_boost_round = int(params['nrounds'])

    model_params = {
        'eta': params['eta'],
        'max_depth': int(params['max_depth']),
        'gamma': params['gamma'],
        'subsample': params['subsample'],
        'colsample_bytree': params['colsample_bytree'],
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'nthread': 4,
        #'alpha': params['alpha']
    }

    model = xgb.train(model_params, dtrain, num_boost_round)
    preds = model.predict(dvalid)
    auc = roc_auc_score(y_valid, preds)
    return {'loss': -auc, 'status': STATUS_OK}

# Run hyperparameter optimization
trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=100, trials=trials)

# Print the best hyperparameters
print(best)




100%|██████████| 100/100 [21:53<00:00, 13.13s/trial, best loss: -0.9366536964980546]
{'colsample_bytree': 0.8060968504244015, 'eta': 0.23920646853418032, 'gamma': 0.49961166881215413, 'max_depth': 7, 'nrounds': 136.0, 'subsample': 0.9884340052122569}


In [ ]:
# Use the best hyperparameters for model training
best_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'eta': best['eta'],
    'max_depth': int(best['max_depth']),
    'gamma': best['gamma'],
    'subsample': best['subsample'],
    'colsample_bytree': best['colsample_bytree'],
    #'alpha' : best['alpha'],
    'scale_pos_weight': scale_pos_weight
}

num_rounds = int(best['nrounds'])

# Training the model
model = xgb.train(best_params, dtrain, num_rounds, [(dtrain, 'train'), (dvalid, 'valid')])

# Predict on validation data
preds = model.predict(dvalid)

# Calculate AUC
auc_score = roc_auc_score(y_valid, preds)
print(f"AUC: {auc_score}")

[0]	train-auc:0.77795	valid-auc:0.76613


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[1]	train-auc:0.81512	valid-auc:0.80815
[2]	train-auc:0.83740	valid-auc:0.82273
[3]	train-auc:0.84112	valid-auc:0.82620
[4]	train-auc:0.85364	valid-auc:0.83798
[5]	train-auc:0.85455	valid-auc:0.83735
[6]	train-auc:0.85866	valid-auc:0.84126
[7]	train-auc:0.86133	valid-auc:0.84454
[8]	train-auc:0.87540	valid-auc:0.85389
[9]	train-auc:0.88824	valid-auc:0.86152
[10]	train-auc:0.89131	valid-auc:0.86419
[11]	train-auc:0.89437	valid-auc:0.86562
[12]	train-auc:0.90013	valid-auc:0.86886
[13]	train-auc:0.90965	valid-auc:0.87608
[14]	train-auc:0.91090	valid-auc:0.87605
[15]	train-auc:0.91364	valid-auc:0.87747
[16]	train-auc:0.91474	valid-auc:0.87693
[17]	train-auc:0.92057	valid-auc:0.88158
[18]	train-auc:0.92350	valid-auc:0.88394
[19]	train-auc:0.92723	valid-auc:0.88572
[20]	train-auc:0.92913	valid-auc:0.88567
[21]	train-auc:0.93020	valid-auc:0.88641
[22]	train-auc:0.93424	valid-auc:0.89028
[23]	train-auc:0.93644	valid-auc:0.89009
[24]	train-auc:0.93858	valid-auc:0.89048
[25]	train-auc:0.94012	va

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_valid, preds)

# Calculate the area under the ROC curve (AUC)
roc_auc = auc(fpr, tpr)

# Find the best threshold based on Youden's J statistic
best_threshold_index = np.argmax(tpr - fpr)
best_threshold = thresholds[best_threshold_index]

# Calculate performance metrics at the best threshold (e.g., accuracy, precision, recall)
binary_preds = (preds > best_threshold).astype(int)

# You can use various metrics to evaluate performance, e.g., accuracy, precision, recall, F1 score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_valid, binary_preds)
precision = precision_score(y_valid, binary_preds)
recall = recall_score(y_valid, binary_preds)
f1 = f1_score(y_valid, binary_preds)

print(f"Best Threshold: {best_threshold:.2f}")

ValueError: ignored

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving cleaned_test 2.csv to cleaned_test 2.csv


In [ ]:
import pandas as pd
import io

model_data_test = pd.read_csv(io.BytesIO(uploaded['cleaned_test 2.csv']))

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving test.csv to test.csv


In [ ]:
import pandas as pd
import io

test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

test.head()

,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,generation,state,zipcode
0,11262,28,blue-collar,single,primary,no,32289,no,yes,cellular,29,feb,1582,29.0,437,1.0,other,Millennials,OK,73125
1,39813,24,entrepreneur,divorced,tertiary,no,1644,no,no,cellular,13,nov,3100,7.0,139,32.0,unknown,Millennials,MI,48902
2,6108,92,technician,NaN,secondary,no,-1035,yes,no,cellular,7,may,3018,4.0,176,14.0,other,Silent Generation,MD,21568
3,43280,24,self-employed,single,secondary,no,9710,no,no,telephone,26,sep,2595,63.0,184,6.0,success,Millennials,SC,29488
4,9011,80,services,single,tertiary,no,81688,no,no,unknown,22,jul,2576,32.0,150,15.0,failure,Silent Generation,LA,70608


In [ ]:
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, f1_score
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Assuming you've defined your training data X, validation data dvalid, and y_valid elsewhere

label_encoders = {}
columns_to_encode = ['default', 'housing', 'loan']
for col in columns_to_encode:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

# Assuming 'model' is your trained XGBoost model

# Predict the probabilities for the validation data
probas_valid = model.predict(dvalid)

# Get the FPR, TPR, and thresholds
fpr, tpr, thresholds = roc_curve(y_valid, probas_valid)

# Compute the F1 scores for each threshold
f1_scores = [f1_score(y_valid, probas_valid > t) for t in thresholds]

# Get the threshold that gives the highest F1 score
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Optimal Threshold = {optimal_threshold}")

# Convert probabilities of validation set to class labels using the optimal threshold
binary_preds_valid = (probas_valid > optimal_threshold).astype(int)

# For the test set:
# Convert the test set using the label encoders
for col, le in label_encoders.items():
    model_data_test[col] = le.transform(model_data_test[col])

test_matrix = xgb.DMatrix(model_data_test)
probas_test = model.predict(test_matrix)
binary_preds_test = (probas_test > optimal_threshold).astype(int)

# Display summaries
print(pd.Series(probas_test).describe())
print(pd.Series(binary_preds_test).value_counts())


# Histogram of predicted probabilities
plt.hist(preds, bins=50, color="skyblue")
plt.title("Histogram of Predicted Probabilities")
plt.xlabel("Predicted Probability of Class 1")
plt.ylabel("Frequency")
plt.show()

# Feature importances
importance = model.get_score(importance_type='weight')
sorted_importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)[:15]
plt.barh([x[0] for x in sorted_importance], [x[1] for x in sorted_importance])
plt.xlabel('Importance')
plt.title('Top 15 Feature Importances')
plt.gca().invert_yaxis()
plt.show()

# Save predictions
model_data_test['predictions'] = binary_preds

output = pd.DataFrame({
    'id': test['id'],
    'cd': model_data_test['predictions']
})

output.to_csv("predictions.csv", index=False)





NameError: ignored